In [121]:
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot as plt

In [13]:
USE_CUDA = torch.cuda.is_available()

# add seed to make sure the result can be reproduced
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)

In [24]:
rating_matrix = torch.rand((200,300), dtype=torch.double) * 5 # 200 users and 300 items

In [25]:
rating_matrix

tensor([[1.6612, 4.1210, 0.3640,  ..., 0.6650, 0.2198, 4.8357],
        [1.5374, 0.6454, 3.7844,  ..., 0.6718, 1.7241, 4.8555],
        [1.3164, 3.8213, 3.7675,  ..., 0.5212, 4.7950, 1.1560],
        ...,
        [2.2500, 0.3971, 3.9112,  ..., 1.5492, 1.6214, 0.0902],
        [0.9362, 4.7794, 0.5447,  ..., 0.8912, 1.5400, 0.5561],
        [3.7536, 2.4913, 2.9639,  ..., 4.7032, 3.6347, 3.5916]],
       dtype=torch.float64)

In [42]:
mask_matrix = (torch.rand((200, 300)) > 0.2).int()

In [43]:
mask_matrix

tensor([[1, 1, 1,  ..., 0, 0, 1],
        [1, 1, 0,  ..., 0, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 0, 1],
        [1, 0, 1,  ..., 0, 0, 1],
        [1, 1, 0,  ..., 1, 0, 1]], dtype=torch.int32)

In [140]:
l = []
learning_rate = 1e-5

In [156]:
num_users = 200
num_items = 300
hidden_size = 30
u = torch.randn(1, requires_grad = True)
bu = torch.randn((num_users, 1), requires_grad = True)
bi = torch.randn((num_items, 1), requires_grad = True)
p = torch.randn((num_users, hidden_size), requires_grad = True)
q = torch.randn((num_items, hidden_size), requires_grad = True)    

In [157]:
for epoch in range(100000):
    
    r_predict = torch.mm(p, q.t()) + bu + bi.t() + u # (num_users, num_items)
    
    loss = torch.sum((r_predict * mask_matrix - rating_matrix * mask_matrix).pow(2)) + 0.1 * (bu.pow(2).sum() + bi.pow(2).sum() + p.pow(2).sum() + q.pow(2).sum())
    if epoch % 1000 == 0:
        print("epoch", epoch, "loss", loss.item())
    
    loss.backward()
    
    with torch.no_grad():
        u -= learning_rate * u.grad
        bu -= learning_rate * bu.grad
        bi -= learning_rate * bi.grad
        p -= learning_rate * p.grad
        q -= learning_rate * q.grad
        u.grad.zero_()
        bu.grad.zero_()
        bi.grad.zero_()
        p.grad.zero_()
        q.grad.zero_()

epoch 0 loss 1706722.4886925956
epoch 1000 loss 91196.75096977068
epoch 2000 loss 82189.63200047139
epoch 3000 loss 78685.08442425553
epoch 4000 loss 75886.85518920128
epoch 5000 loss 73485.60639042467
epoch 6000 loss 71425.21193201157
epoch 7000 loss 69651.54818728345
epoch 8000 loss 68114.6049016031
epoch 9000 loss 66773.89924696015
epoch 10000 loss 65597.21150961949
epoch 11000 loss 64558.55720098704
epoch 12000 loss 63636.838692317855
epoch 13000 loss 62814.934171483896
epoch 14000 loss 62078.962231945734
epoch 15000 loss 61417.616311114914
epoch 16000 loss 60821.60424308446
epoch 17000 loss 60283.1928522414
epoch 18000 loss 59795.85800752274
epoch 19000 loss 59354.027146591834
epoch 20000 loss 58952.87739928913
epoch 21000 loss 58588.190657962674
epoch 22000 loss 58256.24244250346
epoch 23000 loss 57953.7135069488
epoch 24000 loss 57677.63088178211
epoch 25000 loss 57425.32031735996
epoch 26000 loss 57194.37234556438
epoch 27000 loss 56982.61384577702
epoch 28000 loss 56788.087125

In [160]:
def evaluate():
    r_predict = torch.mm(p, q.t()) + bu + bi.t() + u # (num_users, num_items)
    loss = torch.sum((r_predict * (1 - mask_matrix) - rating_matrix * (1-mask_matrix)).pow(2))
    return loss

In [161]:
evaluate()

tensor(45888.5589, dtype=torch.float64, grad_fn=<SumBackward0>)

In [162]:
r_predict * (1 - mask_matrix)

tensor([[0.0000, 0.0000, 0.0000,  ..., 3.3125, 4.0608, 0.0000],
        [0.0000, 0.0000, 2.6633,  ..., 1.4139, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.5819, 0.0000],
        [0.0000, 2.0817, 0.0000,  ..., 0.5112, 2.9696, 0.0000],
        [0.0000, 0.0000, 2.2950,  ..., 0.0000, 3.2472, 0.0000]],
       grad_fn=<MulBackward0>)

In [163]:
rating_matrix * (1-mask_matrix)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.6650, 0.2198, 0.0000],
        [0.0000, 0.0000, 3.7844,  ..., 0.6718, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.6214, 0.0000],
        [0.0000, 4.7794, 0.0000,  ..., 0.8912, 1.5400, 0.0000],
        [0.0000, 0.0000, 2.9639,  ..., 0.0000, 3.6347, 0.0000]],
       dtype=torch.float64)